# Anonymous Challenge

Justin Ferrara

In [1]:
# Import required packages
# If packages do not load on different computer, run !pip install [package]
import os
import pandas as pd
import numpy as np
from datetime import datetime

### Prompt
A company has reached out to you for help creating their very first business metrics. You have been given a randomized dataset containing their mock transaction data. It might require cleaning some messy columns, and you may find some fields are not that useful for your analysis. 

Select the top 3 most important KPIs for this company to measure. Explain to us what they are and why they are important. Create a dashboard around these KPIs that will help the company stay on track.

### Approach

1. Business judgement
2. Data collection
3. Data preparation / cleaning
4. Data analysis
5. Data presentation

### Business judgement

The provided data contains transaction-level detail including user phone number, total spend, approximate location (zip code), timestamp of transaction, etc. Since the mock business has not established any business metrics previously, I will assume understanding historical and current revenue numbers is of high importance. Consequently, I will deconstruct revenue into its component parts to explore how the business can track, monitor, and analyze revenue to identify how revenue may be improved.

* Monthly revenue
    * Average spend per transaction
    * Number of transactions
        * Average number of transactions per user
        * Number of users
            * Number of new users
            * Number of returning users

It is important to not allow available data to limit hypothesis generation when structuring a business case like revenue because new data may be able to be collected. As such, my initial hypothesis for key KPIs may not all be possible with currently available data.

The three metrics chosen for this exercise were (1) *monthly revenue*, (2) *average spend per transaction*, and (3) *monthly transactions*. When considering KPIs for a company, it is important to choose metrics that are important indicators of overall health of the business and to adopt metrics that are mutually exclusive, but collectively exhaustive to allow for clear actionability.

In this scenario, monthly revenue was chosen as a priority KPI as it is indicative of overall business health, yet monthly revenue alone is not as actionable as it could be. By decomposing revenue into its component parts, we see that average spend per transactions and the number of transactions considered together can explain any fluctuations in total revenue. This means that if revenue, let's say, decreases next month, an analyst can look to see if the decrease is driven by lower spending or lower usage. 

These same metrics may be tracked at a different unit of aggregation as well. For example, we can assess revenue and its component parts for each individual spot.

If additional user data were made available, each metric could be broken down even further to arrive at more specific metrics. In this example, the number of transactions can be boiled down to usage by user and the number of users. And so on, as illustrated by the graphic below.

### Data collection

Transaction-level data has been provided in CSV form.

In [2]:
# Set working directory to a folder containing raw data file
os.chdir("/Users/JustinFerrara/Desktop/Anonymous")

In [3]:
raw_data = pd.read_csv('data-cEOUaHISyGUP1JLWJN8i.csv')
raw_data.head(10)

,transaction_id,created_at_UTC,user_phone,amount_paid_cents,zip_code,spot_id
0,1,2022-02-03 08:59:46,(674) 384-2776,9 065,30006,5
1,2,2021-09-02 15:55:02,1-251-843-1844,6 844,23841,5
2,3,2022-05-11 11:04:50,1-451-684-9420,9 446,482219,9
3,4,2021-01-24 17:05:12,1-658-836-8233,660,15537,8
4,5,2020-11-22 08:13:02,(481) 946-2170,5 714,221573,11
5,6,2021-08-27 16:24:06,(771) 647-7413,8 822,66352,3
6,7,2022-10-10 13:38:37,1-928-938-1403,4 526,354135,5
7,8,2022-07-18 02:22:57,1-903-517-7431,5 284,563958,8
8,9,2021-08-26 14:12:47,(774) 472-3583,3 413,41744-248,11
9,10,2021-12-17 18:35:16,(392) 515-2956,182,38402,9


In [4]:
raw_data.dtypes

transaction_id        int64
created_at_UTC       object
user_phone           object
amount_paid_cents    object
zip_code             object
spot_id               int64
dtype: object

### Data preparation / cleaning
After loading the raw CSV file and previewing the dataset, we immediately see a few things that may need attention for our analysis. Initial observations include:

* created_at_UTC is a timestamp field stored in Coordinated Universal Time (UTC) standard. This means, a local timezone is needed in order to conduct any analysis on transactions by time of day.
* The user_phone field appears to contain phone numbers of variable format. Phone numbers will need to be standardized before incorporating this field into an analysis.
* amount_paid_cents appears to use a space character possibly in place of a decimal point or a comma. Clarification is needed.
* The zip_code field is inconsistent -- some data points are in standard five character format, some include 4 digit codes to further specify location, some have anywhere from 5-6 or more digits.

Let's start by cleaning up the date-time field.

In [5]:
# Convert to datetime data type
raw_data['created_at_UTC'] = pd.to_datetime(raw_data['created_at_UTC'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Create new fields for month, year
raw_data['year'] = pd.to_datetime(raw_data['created_at_UTC']).dt.strftime('%Y')
raw_data['month'] = pd.to_datetime(raw_data['created_at_UTC']).dt.strftime('%m')

raw_data['year_month'] = raw_data['year'] + raw_data['month']

raw_data.head(10)

,transaction_id,created_at_UTC,user_phone,amount_paid_cents,zip_code,spot_id,year,month,year_month
0,1,2022-02-03 08:59:46,(674) 384-2776,9 065,30006,5,2022,02,202202
1,2,2021-09-02 15:55:02,1-251-843-1844,6 844,23841,5,2021,09,202109
2,3,2022-05-11 11:04:50,1-451-684-9420,9 446,482219,9,2022,05,202205
3,4,2021-01-24 17:05:12,1-658-836-8233,660,15537,8,2021,01,202101
4,5,2020-11-22 08:13:02,(481) 946-2170,5 714,221573,11,2020,11,202011
5,6,2021-08-27 16:24:06,(771) 647-7413,8 822,66352,3,2021,08,202108
6,7,2022-10-10 13:38:37,1-928-938-1403,4 526,354135,5,2022,10,202210
7,8,2022-07-18 02:22:57,1-903-517-7431,5 284,563958,8,2022,07,202207
8,9,2021-08-26 14:12:47,(774) 472-3583,3 413,41744-248,11,2021,08,202108
9,10,2021-12-17 18:35:16,(392) 515-2956,182,38402,9,2021,12,202112


Next, we can standardize the user_phone field.

In [6]:
# Remove all non-digit characters join the separated character list back into one string
raw_data['user_phone_edited'] = raw_data['user_phone'].apply(lambda x: filter(str.isdigit, x))
raw_data['user_phone_edited'] = raw_data['user_phone_edited'].apply(lambda x: "".join(x))

# Take only final 10 characters of phone number to remove any prefix digits
raw_data['user_phone_edited'] = raw_data['user_phone_edited'].apply(lambda x: x[-10:])

raw_data.head(10)

,transaction_id,created_at_UTC,user_phone,amount_paid_cents,zip_code,spot_id,year,month,year_month,user_phone_edited
0,1,2022-02-03 08:59:46,(674) 384-2776,9 065,30006,5,2022,02,202202,6743842776
1,2,2021-09-02 15:55:02,1-251-843-1844,6 844,23841,5,2021,09,202109,2518431844
2,3,2022-05-11 11:04:50,1-451-684-9420,9 446,482219,9,2022,05,202205,4516849420
3,4,2021-01-24 17:05:12,1-658-836-8233,660,15537,8,2021,01,202101,6588368233
4,5,2020-11-22 08:13:02,(481) 946-2170,5 714,221573,11,2020,11,202011,4819462170
5,6,2021-08-27 16:24:06,(771) 647-7413,8 822,66352,3,2021,08,202108,7716477413
6,7,2022-10-10 13:38:37,1-928-938-1403,4 526,354135,5,2022,10,202210,9289381403
7,8,2022-07-18 02:22:57,1-903-517-7431,5 284,563958,8,2022,07,202207,9035177431
8,9,2021-08-26 14:12:47,(774) 472-3583,3 413,41744-248,11,2021,08,202108,7744723583
9,10,2021-12-17 18:35:16,(392) 515-2956,182,38402,9,2021,12,202112,3925152956


Next, the amount_paid_cents field needs to be cleaned then converted to a numeric field.

Per instruction from challenge proctors, I have replaced the spaces in the amount_paid_cents field with a decimal point. For example, 9 065 will become 9.065 cents.

In [7]:
# Replace the empty space within data points then convert to numeric type for future calculations
raw_data['amount_paid_cents_edited'] = raw_data['amount_paid_cents'].str.replace(' ', '.',regex=True)
raw_data['amount_paid_cents_edited'] = pd.to_numeric(raw_data['amount_paid_cents_edited'])

# Convert cents to dollars for better interpretability
raw_data['amount_paid_dollars'] = raw_data['amount_paid_cents_edited']/100

raw_data.head(10)

,transaction_id,created_at_UTC,user_phone,amount_paid_cents,zip_code,spot_id,year,month,year_month,user_phone_edited,amount_paid_cents_edited,amount_paid_dollars
0,1,2022-02-03 08:59:46,(674) 384-2776,9 065,30006,5,2022,02,202202,6743842776,9.065,0.09065
1,2,2021-09-02 15:55:02,1-251-843-1844,6 844,23841,5,2021,09,202109,2518431844,6.844,0.06844
2,3,2022-05-11 11:04:50,1-451-684-9420,9 446,482219,9,2022,05,202205,4516849420,9.446,0.09446
3,4,2021-01-24 17:05:12,1-658-836-8233,660,15537,8,2021,01,202101,6588368233,660.000,6.60000
4,5,2020-11-22 08:13:02,(481) 946-2170,5 714,221573,11,2020,11,202011,4819462170,5.714,0.05714
5,6,2021-08-27 16:24:06,(771) 647-7413,8 822,66352,3,2021,08,202108,7716477413,8.822,0.08822
6,7,2022-10-10 13:38:37,1-928-938-1403,4 526,354135,5,2022,10,202210,9289381403,4.526,0.04526
7,8,2022-07-18 02:22:57,1-903-517-7431,5 284,563958,8,2022,07,202207,9035177431,5.284,0.05284
8,9,2021-08-26 14:12:47,(774) 472-3583,3 413,41744-248,11,2021,08,202108,7744723583,3.413,0.03413
9,10,2021-12-17 18:35:16,(392) 515-2956,182,38402,9,2021,12,202112,3925152956,182.000,1.82000


Our KPIs will not make use of the zip_code field, so we will leave this field un-cleaned for now.

Finally, we can drop all columns not necessary for our analysis.

In [8]:
data = raw_data[['transaction_id', 'created_at_UTC', 'year_month', 'user_phone_edited', 'amount_paid_dollars', 'spot_id']]
data.head(10)

,transaction_id,created_at_UTC,year_month,user_phone_edited,amount_paid_dollars,spot_id
0,1,2022-02-03 08:59:46,202202,6743842776,0.09065,5
1,2,2021-09-02 15:55:02,202109,2518431844,0.06844,5
2,3,2022-05-11 11:04:50,202205,4516849420,0.09446,9
3,4,2021-01-24 17:05:12,202101,6588368233,6.60000,8
4,5,2020-11-22 08:13:02,202011,4819462170,0.05714,11
5,6,2021-08-27 16:24:06,202108,7716477413,0.08822,3
6,7,2022-10-10 13:38:37,202210,9289381403,0.04526,5
7,8,2022-07-18 02:22:57,202207,9035177431,0.05284,8
8,9,2021-08-26 14:12:47,202108,7744723583,0.03413,11
9,10,2021-12-17 18:35:16,202112,3925152956,1.82000,9


In [9]:
data.groupby(['year_month']).agg({'transaction_id':'count', 'created_at_UTC':np.max}).reset_index()

,year_month,transaction_id,created_at_UTC
0,202011,16,2020-11-30 22:01:12
1,202012,19,2020-12-30 12:53:46
2,202101,32,2021-01-31 19:34:53
3,202102,14,2021-02-28 22:55:54
4,202103,24,2021-03-30 05:40:59
5,202104,16,2021-04-30 18:28:26
6,202105,22,2021-05-31 09:54:22
7,202106,22,2021-06-30 16:53:55
8,202107,16,2021-07-29 14:15:51
9,202108,22,2021-08-31 16:01:08


It appears as though November 2022 only contains half a month of data, so we will exclude the incomplete month from our analyses of monthly metrics.

In [10]:
data = data[data['year_month'] != "202211"]

### Data analysis

Below, I conducted a check to see how many users conducted transactions. I discovered that no user in the dataset performs more than 1 transaction, which means any metric on number of new users or average transaction spend by user would be meaningless metrics in this case.

In [11]:
# How many distinct users are in the dataset?
len(set(data['user_phone_edited'])) == len(set(data['transaction_id']))

True

In [12]:
# Aggregate number of transactions and total spend by month
agg_data = data.groupby(['year_month']).agg({'transaction_id':'count', 'amount_paid_dollars': 'sum'}).reset_index()
agg_data.columns = ['year_month', 'num_transactions', 'total_spend']

# Calculate average spend per transaction
agg_data['avg_spend_per_trans'] = round(agg_data['total_spend'] / agg_data['num_transactions'], 2)

agg_data.head(10)

,year_month,num_transactions,total_spend,avg_spend_per_trans
0,202011,16,9.80316,0.61
1,202012,19,9.91056,0.52
2,202101,32,16.87573,0.53
3,202102,14,0.69340,0.05
4,202103,24,12.75742,0.53
5,202104,16,11.54982,0.72
6,202105,22,10.98820,0.50
7,202106,22,3.74562,0.17
8,202107,16,0.92002,0.06
9,202108,22,16.34858,0.74


In [13]:
# Smooth the dataset by generating a 3-month average
agg_data['num_transactions_3ma'] = agg_data.iloc[:,1].rolling(window=3).mean()
agg_data['total_spend_3ma'] = agg_data.iloc[:,2].rolling(window=3).mean()
agg_data['avg_spend_per_trans_3ma'] = agg_data.iloc[:,3].rolling(window=3).mean()

# Measure growth by calculating the percent change in the 3-month averages
agg_data['num_transactions_3ma_growth'] = agg_data['num_transactions_3ma'].diff()
agg_data['total_spend_3ma_growth'] = agg_data['total_spend_3ma'].diff()
agg_data['avg_spend_per_trans_3ma_growth'] = agg_data['avg_spend_per_trans_3ma'].diff()

agg_data['num_transactions_3ma_growth_perc'] = round(agg_data['num_transactions_3ma_growth'] / agg_data['num_transactions_3ma'], 3)
agg_data['total_spend_3ma_growth_perc'] = round(agg_data['total_spend_3ma_growth'] / agg_data['total_spend_3ma'], 3)
agg_data['avg_spend_per_trans_3ma_growth_perc'] = round(agg_data['avg_spend_per_trans_3ma_growth'] / agg_data['avg_spend_per_trans_3ma'], 3)

agg_data = agg_data[['year_month', 'num_transactions_3ma', 'num_transactions_3ma_growth_perc', 'total_spend_3ma', 'total_spend_3ma_growth_perc', 'avg_spend_per_trans_3ma', 'avg_spend_per_trans_3ma_growth_perc']]

agg_data.head(10)


,year_month,num_transactions_3ma,num_transactions_3ma_growth_perc,total_spend_3ma,total_spend_3ma_growth_perc,avg_spend_per_trans_3ma,avg_spend_per_trans_3ma_growth_perc
0,202011,NaN,NaN,NaN,NaN,NaN,NaN
1,202012,NaN,NaN,NaN,NaN,NaN,NaN
2,202101,22.333333,NaN,12.196483,NaN,0.553333,NaN
3,202102,21.666667,-0.031,9.159897,-0.332,0.366667,-0.509
4,202103,23.333333,0.071,10.108850,0.094,0.370000,0.009
5,202104,18.000000,-0.296,8.333547,-0.213,0.433333,0.146
6,202105,20.666667,0.129,11.765147,0.292,0.583333,0.257
7,202106,20.000000,-0.033,8.761213,-0.343,0.463333,-0.259
8,202107,20.000000,0.000,5.217947,-0.679,0.243333,-0.904
9,202108,20.000000,0.000,7.004740,0.255,0.323333,0.247


We will now construct a dataset based on an aggregation of each spot by month. To do so, we will jump back to the 'data' dataframe created in the previous data prep step.

In [14]:
# Unique number of spots
len(set(data['spot_id']))

15

In [15]:
# Aggregate number of transactions and total spend by month and spot id
spot_data = data.groupby(['year_month', 'spot_id']).agg({'transaction_id':'count', 'amount_paid_dollars': 'sum'}).reset_index()
spot_data.columns = ['year_month', 'spot_id', 'num_transactions', 'total_spend']

# Calculate average spend per transaction
spot_data['avg_spend_per_trans'] = round(spot_data['total_spend'] / spot_data['num_transactions'], 2)

spot_data.head(10)

,year_month,spot_id,num_transactions,total_spend,avg_spend_per_trans
0,202011,1,1,0.01240,0.01
1,202011,2,1,0.04280,0.04
2,202011,3,1,0.07412,0.07
3,202011,4,2,3.36180,1.68
4,202011,6,1,0.05261,0.05
5,202011,7,2,0.07104,0.04
6,202011,8,3,5.86675,1.96
7,202011,11,2,0.14969,0.07
8,202011,13,1,0.06310,0.06
9,202011,14,2,0.10885,0.05


In [16]:
spot_data = spot_data.sort_values(['spot_id', 'year_month'])

spot_data['num_transactions_change'] = spot_data['num_transactions'].diff()
spot_data['total_spend_change'] = spot_data['total_spend'].diff()
spot_data['avg_spend_per_trans_change'] = spot_data['avg_spend_per_trans'].diff()

spot_data.head(10)

,year_month,spot_id,num_transactions,total_spend,avg_spend_per_trans,num_transactions_change,total_spend_change,avg_spend_per_trans_change
0,202011,1,1,0.01240,0.01,NaN,NaN,NaN
10,202012,1,1,0.02424,0.02,0.0,0.01184,0.01
21,202101,1,3,0.20783,0.07,2.0,0.18359,0.05
41,202103,1,1,3.04000,3.04,-2.0,2.83217,2.97
76,202106,1,1,0.02966,0.03,0.0,-3.01034,-3.01
87,202107,1,2,0.09244,0.05,1.0,0.06278,0.02
97,202108,1,1,0.06981,0.07,-1.0,-0.02263,0.02
109,202109,1,1,0.03831,0.04,0.0,-0.03150,-0.03
121,202110,1,1,0.02155,0.02,0.0,-0.01676,-0.02
131,202111,1,1,0.01153,0.01,0.0,-0.01002,-0.01


In [17]:
# Identify areas of overlap when the previous record represents a different spot_id
mask = spot_data['spot_id'] != spot_data['spot_id'].shift(1)

# For records that match the above criteria, replace data point with a NaN
spot_data['num_transactions_change'][mask] = np.nan
spot_data['total_spend_change'][mask] = np.nan
spot_data['avg_spend_per_trans_change'][mask] = np.nan

# Clean up dataset by reorganizing columns
spot_data = spot_data[['year_month', 'spot_id', 'total_spend', 'total_spend_change', 'avg_spend_per_trans', 'avg_spend_per_trans_change', 'num_transactions', 'num_transactions_change']]
spot_data = spot_data.sort_values(by=['year_month', 'spot_id'], ascending=(False, True))

spot_data.head(10)

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,year_month,spot_id,total_spend,total_spend_change,avg_spend_per_trans,avg_spend_per_trans_change,num_transactions,num_transactions_change
252,202210,1,0.07858,0.04440,0.08,0.05,1,0.0
253,202210,2,3.18000,3.03531,3.18,3.13,1,-2.0
254,202210,3,0.14607,0.05034,0.07,0.02,2,0.0
255,202210,4,0.13167,0.05830,0.07,0.00,2,1.0
256,202210,5,0.18663,0.15086,0.05,0.01,4,3.0
257,202210,7,0.01918,-0.08626,0.02,-0.03,1,-1.0
258,202210,8,0.07239,-0.16996,0.07,-0.01,1,-2.0
259,202210,9,0.02527,-0.07533,0.03,-0.02,1,-1.0
260,202210,10,0.08814,0.05708,0.09,0.06,1,0.0
261,202210,11,0.07277,-3.89705,0.07,-1.25,1,-2.0


### Data presentation

Output new prepared data files as JSON for ingestion in Observable.

In [18]:
# Convert DataFrames to JSON format
data.to_json('transaction_data.json', orient='records')
agg_data.to_json('revenue_metrics.json', orient='records')
spot_data.to_json('spot_metrics.json', orient='records')

A web-based dashboard has been built in Observable using JavaScript and D3.